# AFSK Demodulator on PYNQ

This is a series of notebooks on migrating an [AFSK demodulator we developed in Python](https://github.com/mobilinkd/afsk-demodulator/blob/master/afsk-demodulator.ipynb) to FPGA.  This notebook is designed to be run on the Pynq board itself and links to the Pynq-specific components.  This is a companion to the [other notebook](afsk-demodulator-fpga.ipynb) which is designed to guide the reader through the creation of the IP components in Vivado and Vivado HLS.

![PYNQ Z2 board](03_PYNQ-Z2.png)

The development is being done on a [PYNQ Z2 board](http://www.tul.com.tw/ProductsPYNQ-Z2.html), which uses a Xilinx Zync 7020 SOC.  This chip integrates Xilinx Artix-7 FPGA fabric with a dual-core ARM Cortex A9 processor.  The PYNQ boards are designed to run **Linux** (based on Ubuntu), **Python**, and **Jupyter Notebooks**.  Code developed for the FPGA fabric can be instantiated and accessed directly from Python.  More information on Pynq can be found here: http://www.pynq.io, and the Pynq software stack is documented here: https://pynq.readthedocs.io/

The purpose of this project is to teach myself FPGA development, and to share that process with others.  This is not meant as a "getting started" tutorial, but rather as a step for me, after having done a few basic tutorials, to apply an area that I am quite knowledgable in to one that I am learning about.

The PYNQ Z2 has been a great board for teaching me, a software engineer, how to develop for FPGA.  And the PYNQ environment is an amazing platform for teaching FPGA development to software engineers.  Both are highly recommended.

The PYNQ development environment will allow us to move the demodulator components, step by step, into the FPGA.  This will allow us to explore new areas of the development process and test our code in small, digestible steps.

I will be using this as a *coding kata*, which is to apply repetition to coding practices so that one becomes proficient -- and eventually expert -- in the process and tools used to do development.  I am aiming for proficiency in Vivado and Vivado HLS at this point.

### Post Script

After going through this process I do feel much more comfortable Vivado HLS, Vivado, and Pynq.  I have no problem starting a project from scratch, building and testing in Vivado HLS, creating a block design and integrating the IP in a project, and creating the Python infrastructure it on Pynq.  There is still a lot to learn on the Zynq system itself.


## Getting Started

For those who would like to follow along with this project to learn FPGA development, please start with the other top-level notebook.  It will guide you through the IP creation process.

If, instead, you would like to play around with the pre-built components of the demodulator on Pynq, you can use this notebook as a jumping off point between the various components.

You will need:

 - A Zynq development board running Pynq
 - Experience with the tutorials on the Pynq web site
 

## Development Steps

We will be migrating the demodulator step by step, in the same steps shown in the Python code linked above.  This will give us good practice going through the steps required to generate and use IP for the FPGA.  This repetition help me to learn the mechanics of a process, which is different than understanding the just the concepts.

This project will require that we get into AXI Streams and DMA very quickly, as we will be transferring blocks of audio data to the FPGA for processing, then reading the processed data back. We will also start using interrupts and asyncio, as well as wiring up LEDs to the FPGA.

To start, we will create a basic project that accepts an input stream and returns an output stream.  It will perform a very basic operation to prove that it is actually working.  We will then use that as the basis for further work.  We will be moving each piece, having a comlete working demodulator after each step, but with ever more work being done in the FPGA.

We will:

 1. [Create a basic AXI Streaming solution](intro/Pynq/afsk-demodulator-pynq.ipynb).
    1. We will use this as the basis for further work as we will initially be streaming data in and out of the FPGA symmetrically (same number of elements out and in).
 1. [Migrate the band pass filter to FPGA](band_pass_filter/Pynq/afsk-demodulator-pynq.ipynb).
    1. Implement a FIR filter in C++ for the FPGA.
 1. [Move the Correlator into the FPGA](correlator/Pynq/afsk-demodulator-pynq.ipynb).
    1. Move the comparator into the FPGA.
    1. Move the delay line into the FPGA.
    1. Move the XOR logic to the FPGA.
 1. [Move the low-pass filter to the FPGA](low_pass_filter/Pynq/afsk-demodulator-pynq.ipynb).
    1. Move the low-pass filter to the FPGA.
    1. Move the second comparator into the FPGA.
 1. [FPGA I/O Interlude](io_interlude/Pynq/afsk-demodulator-pynq.ipynb).
    1 How to access buttons and LEDs from the PL fabric, independent of the PS.
    1 How to use asyncio to capture interrupts from the PL.
 1. [Move the digital PLL into the FPGA](digital_pll/Pynq/afsk-demodulator-pynq.ipynb).
    1. Move the IIR filter into the FPGA.
    1. Move the hysteresis code into the FPGA.
    1. Add a DCD indicator to the project, using a LED from the Pynq Z2 board.
 1. [Move the HDLC code into the FPGA](hdlc/Pynq/afsk-demodulator-pynq.ipynb).
    1. Move the NRZI code to FPGA.
    1. Move the CRC code into the FPGA.
 1. [Complete the demodulator](demodulator/Pynq/afsk-demodulator-pynq.ipynb).
    1. Tie all of the components together.
    1. Test the code with a complete track of data.


We will leave the AX.25 decoding logic in Python.  But we may add a serial port to our code and add KISS and SLIP encoders to the project so the output can be consumed by Linux AX.25 components.

## Future Work

The Pynq boards have high-resolution audio input and output capability -- 24-bit, 96kbps -- and the audio chip itself is capable of DSP operations such as filtering, AGC, etc.  The APSoC itself has a 1Msps ADC on the PL side, and a large number of I/Os.  Implementing a full 1200 baud TNC that can interface to a radio should be possible.  If that is done, this board could make for an interesting iGate or Digipeater.  A small add-on board might be needed to implement the PTT circuitry.